## TP2 : Classification using Linear & Quadratic Discriminant Analysis

First think of configuring your notebook :

In [ ]:
import csv
# import os
from pylab import *
import numpy as np
from numpy import linalg as la


## Reading synthetic data
Load the training and test data sets |synth_train.txt| and
|synth_test.txt| already used for Knn. Targets belong to {1,2} and entries belong to R^2.
We have 100 training data samples and 200 test samples.

* the 1st column contains the label of the class the sample, 
* columns 2 & 3 contain the coordinates of each sample in 2D.

In [ ]:
train = np.loadtxt('synth_train.txt')

test = np.loadtxt('synth_test.txt')

## Recall about the main steps of discriminant analysis:
* estimation of weights `pi_1` and `pi_2` for each class,
* estimation of empirical means `mu_1` and `mu_2` for each class, 
* estimation of empirical covariance matrices  `sigma_1` and `sigma_2`,
* computation of the common averaged covariance `sigma` (average of intra-class covariances),
* computation of log-probabilities of belonging to each class,
* decision of classification,
* display results.


## TO DO : linear & quadratic discriminant analysis (LDA & QDA)
1. Implement a classifier using LDA of the data set. 
2. Then implement QDA classification.
3. In each case (LDA & QDA) show the decision boundary and
compute the error rate respectively for the training set and the test set. 
4. Compare and comment on your results with LDA and QDA.
5. You may also compare your results to K nearest neighbours.

_Indication 1 : matrices `sigma` are of size 2x2.
More generally, be careful of the sizes of vectors and matrices you
manipulate._

_Indication 2 : to display the regions of decision, you may use:_


In [ ]:
Nx1=100 # number of samples for display
Nx2=100
x1=np.linspace(-2.5,1.5,Nx1)  # sampling of the x1 axis 
x2=np.linspace(-0.5,3.5,Nx2)  # sampling of the x2 axis
[X1,X2]=np.meshgrid(x1,x2)  
x=np.hstack((X1.flatten('F'),X2.flatten('F'))) # list of the coordinates of points on the grid
#N = size(x,axis=0)

# Then compute the sampled prediction class_L for each couple (X1,X2)

In [ ]:
from tkinter import Y
import pandas as pd

class LDA():
    def __init__(self, train, test):
        self.train_df = pd.DataFrame(train,columns = ['classe', 'x1', 'x2'])
        self.test_df = pd.DataFrame(test,columns = ['classe', 'x1', 'x2'])
        
        
    def get_pi_estimators(self):
        return [pi for pi in self.train_df.classe.value_counts(normalize=True, ascending=True).values]
    
    def get_mu_estimators(self):
        classes = self.train_df.classe.unique().tolist()
        mu = np.zeros((len(classes), 2))
        for i, c in enumerate(classes):
            mu[i] = self.train_df[self.train_df.classe==c][['x1', 'x2']].sum(axis=0).to_numpy() / self.train_df[self.train_df.classe==c].shape[0]
        return mu
    
    def get_sigma_estimators(self):
        mu = self.get_mu_estimators()
        classes = self.train_df.classe.unique().tolist()
        sigma_moy = np.zeros((2,2))
        for i, c in enumerate(classes):
            train_df_c = self.train_df[self.train_df.classe==c]
            sigma = np.zeros((2,2))
            for j in range(train_df_c.shape[0]):
                xn = train_df_c[['x1', 'x2']].iloc[j].to_numpy().reshape((2,1))
                sigma += ( train_df_c[['x1', 'x2']].iloc[j].to_numpy().reshape((2,1)) - mu[i].reshape((2,1)) ) @ ( train_df_c[['x1', 'x2']].iloc[j].to_numpy().reshape((2,1)) - mu[i].reshape((2,1)) ).T
            sigma_moy += sigma
        
        return sigma_moy/self.train_df.shape[0]
    
    def get_log_probabilities(self, df):
        pi = self.get_pi_estimators()
        mu = self.get_mu_estimators()
        sigma = self.get_sigma_estimators()
        prediction = np.zeros((len(df), mu.shape[0]))
        for i in range(df.shape[0]):
            x = df[['x1', 'x2']].iloc[i].to_numpy().reshape((2,1))
            y = np.zeros(mu.shape[0])
            for j in range(mu.shape[0]):
                y[j] = np.log(pi[j]) + x.T @ np.linalg.inv(sigma) @ mu[j].reshape((2,1)) - 1/2 * mu[j].reshape((2,1)).T @ np.linalg.inv(sigma) @ mu[j].reshape((2,1))
            prediction[i] = y
        return prediction
    def classification(self, train=True):
        if train:
            df = self.train_df
        else:
            df = self.test_df
        prediction = self.get_log_probabilities(df)
        return np.argmin(prediction, axis=1)
        
            
        

In [ ]:
class QDA(LDA):
    def __init__(self, train, test):
        super().__init__(train, test)
        
    
    def get_sigma_estimators(self):
        mu = self.get_mu_estimators()
        classes = self.train_df.classe.unique().tolist()
        sigma_list =[]
        for i, c in enumerate(classes):
            train_df_c = self.train_df[self.train_df.classe==c]
            sigma = np.zeros((2,2))
            for j in range(train_df_c.shape[0]):
                xn = train_df_c[['x1', 'x2']].iloc[j].to_numpy().reshape((2,1))
                sigma += ( train_df_c[['x1', 'x2']].iloc[j].to_numpy().reshape((2,1)) - mu[i].reshape((2,1)) ) @ ( train_df_c[['x1', 'x2']].iloc[j].to_numpy().reshape((2,1)) - mu[i].reshape((2,1)) ).T
            sigma_list.append(sigma/train_df_c.shape[0])
        
        return sigma_list
    
    def get_log_probabilities(self, df):
        pi = self.get_pi_estimators()
        mu = self.get_mu_estimators()
        sigma = self.get_sigma_estimators()
        prediction = np.zeros((len(df), mu.shape[0]))
        for i in range(df.shape[0]):
            x = df[['x1', 'x2']].iloc[i].to_numpy().reshape((2,1))
            y = np.zeros(mu.shape[0])
            for j in range(mu.shape[0]):
                y[j] = np.log(pi[j]) - 1/2 * np.log(np.linalg.det(sigma[j])) -1/2 *  (x - mu[j].reshape((2,1))).T @ np.linalg.inv(sigma[j]) @ (x - mu[j].reshape((2,1)))
            prediction[i] = y
        return prediction

## TO DO : LDA & QDA using scikit-learn module

The module `scikit-learn` is dedicated to machine learning algorithms. Many of them are available in a simple manner. For LDA and QDA, have a look at the tutorial available at http://scikit-learn.org/stable/modules/lda_qda.html 

**Warning** : you may have a critical view of the way LDA and QDA are illustrated in the proposed example...


